In [15]:
import pandas as pd
import numpy as np

In [4]:
import pyranges

In [24]:
data = pd.read_table("data/pchic/0071_Blood - Non-activated CD4+ Cells_merged_loop.txt")
standard_size = 5_000
def expand_enrich_pchic(df):
    df[["bait_frag_chr", "bait_frag_start", "bait_frag_end"]] = df["bait_frag"].str.split(",", expand=True)
    df[["other_frag_chr", "other_frag_start", "other_frag_end"]] = df["other_frag"].str.split(",", expand=True)
    df["bait_size"] = np.abs(df["bait_frag_start"].apply(int) - df["bait_frag_end"].apply(int))
    df["other_size"] = np.abs(df["other_frag_start"].apply(int) - df["other_frag_end"].apply(int))
    df["bait_frag_midpoint"] = df.apply(lambda x: int((int(x["bait_frag_start"]) + int(x["bait_frag_end"])) / 2), axis=1)
    df["bait_frag_standard_size_start"] = df["bait_frag_midpoint"] - standard_size
    df["bait_frag_standard_size_end"]   = df["bait_frag_midpoint"] + standard_size - 1
    df["other_frag_midpoint"] = df.apply(lambda x: int((int(x["other_frag_start"]) + int(x["other_frag_end"])) / 2), axis=1)
    df["other_frag_standard_size_start"] = df["other_frag_midpoint"] - standard_size
    df["other_frag_standard_size_end"]   = df["other_frag_midpoint"] + standard_size - 1
    negative_starting = df.loc[df["other_frag_standard_size_start"] <= 0, "other_frag_standard_size_start"]
    negative_ending = df.loc[df["other_frag_standard_size_start"] <= 0, "other_frag_standard_size_end"]
    df.loc[negative_starting.index, "other_frag_standard_size_end"] = (negative_ending - negative_starting + 1).apply(int)
    df.loc[negative_starting.index, "other_frag_standard_size_start"] = 1
    return df
data = expand_enrich_pchic(data)

In [20]:
for bait in data["bait_frag"].unique():
    pass

In [37]:
bait = data.sample(1)["bait_frag"].values[0]

In [41]:
bait_midpoint = data[data["bait_frag"] == bait]["bait_frag_midpoint"].values[0]Chromosome

In [79]:
others = data[data["bait_frag"] == bait]["other_frag_midpoint"]

other_std = pd.concat((others, pd.Series(bait_midpoint))).std()
other_n = len(others)

In [80]:
candidate_midpoint = np.random.normal(bait_midpoint, other_std, size=other_n*2)
candidate_midpoint = candidate_midpoint.astype(int)
candidate_midpoint

array([95734435, 95316936, 95614445, 96765648, 93732257, 96019066,
       96283977, 96331200, 96434880, 96193883, 95721419, 95596280,
       96415069, 95152214, 95526647, 95884855, 95921282, 96313710,
       96857853, 96858980, 96147282, 96916181, 96787979, 96593356,
       97770523, 96073189, 97372893, 96471953, 95942016, 94803405,
       96123730, 95827995, 95073850, 94895105, 95684748, 95288889,
       97102815, 95159383, 94961418, 97304001, 96742266, 95198637,
       95480440, 95660227, 95719984, 96667888, 96792044, 95057618,
       96200510, 95644687, 95696459, 96565955, 95625416, 96092826,
       96609572, 96177434, 96308747, 95228012, 96133463, 96596297,
       96334163, 97346696, 96829590, 96517702, 95213158, 96779534,
       96373672, 96960000, 96044352, 96685475, 95662180, 96468856,
       97168492, 95963060, 96793450, 97180515, 96364868, 95773168,
       95660967, 94998660, 96228663, 95010897, 96733921, 96557384,
       95332448, 96907891, 95103118, 95875646, 95857040, 96650

In [81]:
generated = pyranges.PyRanges(pd.DataFrame({"Chromosome": bait.split(",")[0], "Start": candidate_midpoint - 5_000, "End": candidate_midpoint + 5_000}))

/home/kacper/.pyenv/versions/3.11.8/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [82]:
actual = pyranges.PyRanges(data[data["bait_frag"] == bait][["other_frag_chr", "other_frag_standard_size_start", "other_frag_standard_size_end"]].rename(
    columns = {
    "other_frag_chr": "Chromosome", "other_frag_standard_size_start": "Start", "other_frag_standard_size_end": "End"
    }
))

/home/kacper/.pyenv/versions/3.11.8/lib/python3.11/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [86]:
generated = generated.subtract(actual)

In [94]:
reduced = generated.df[:other_n]
reduced = reduced["Chromosome"].astype(str) + "," + reduced["Start"].astype(str) + "," + reduced["End"].astype(str)

In [114]:
generated[(generated.df["End"] < hg.sizes["chr5"]) & (generated.df["Start"] > 0)]

,Chromosome,Start,End
0,chr5,95729435,95739435
1,chr5,95311936,95321936
2,chr5,95609445,95619445
3,chr5,96760648,96770648
4,chr5,93727257,93737257
...,...,...,...
137,chr5,96431005,96441005
138,chr5,96933442,96943442
139,chr5,96000024,96010024
140,chr5,96901662,96911662


In [96]:
pd.DataFrame({
    "bait_frag" : bait,
    "other_frag": reduced
})

,bait_frag,other_frag
0,"chr5,96210582,96211797","chr5,95729435,95739435"
1,"chr5,96210582,96211797","chr5,95311936,95321936"
2,"chr5,96210582,96211797","chr5,95609445,95619445"
3,"chr5,96210582,96211797","chr5,96760648,96770648"
4,"chr5,96210582,96211797","chr5,93727257,93737257"
...,...,...
75,"chr5,96210582,96211797","chr5,95870646,95880646"
76,"chr5,96210582,96211797","chr5,95852040,95862040"
77,"chr5,96210582,96211797","chr5,96645126,96655126"
78,"chr5,96210582,96211797","chr5,96065835,96075835"


In [132]:
EXTRA_SAMPLES = 5
import genomepy
hg = genomepy.Genome("hg19")

def generate_random_negatives(data, bait):
    data_focused = data[data["bait_frag"] == bait]
    bait_midpoint = data_focused["bait_frag_midpoint"].values[0]
    chr = bait.split(",")[0]

    others = data_focused["other_frag_midpoint"]
    
    other_std = pd.concat((others, pd.Series(bait_midpoint))).std() # avoid situations of 1 other
    other_n = len(others)

    candidate_midpoint = np.random.normal(bait_midpoint, other_std, size=other_n*2 + EXTRA_SAMPLES)
    candidate_midpoint = candidate_midpoint.astype(int)

    generated = pyranges.PyRanges(pd.DataFrame({
        "Chromosome": chr, 
        "Start": candidate_midpoint - standard_size, 
        "End": candidate_midpoint + standard_size
    }))

    actual = pyranges.PyRanges(
        data_focused[["other_frag_chr", "other_frag_standard_size_start", "other_frag_standard_size_end"]]
        .rename(columns = {
            "other_frag_chr": "Chromosome", 
            "other_frag_standard_size_start": "Start", 
            "other_frag_standard_size_end": "End"
        })
    )

    filtered_overlaps = generated.subtract(actual)
    filtered_overruns = filtered_overlaps.df[(filtered_overlaps.df["End"] < hg.sizes[chr]) & (filtered_overlaps.df["Start"] > 0)]
    reduced = filtered_overruns[:other_n + EXTRA_SAMPLES]
    reduced = reduced["Chromosome"].astype(str) + "," + reduced["Start"].astype(str) + "," + reduced["End"].astype(str)

    return pd.DataFrame({
        "bait_frag" : bait,
        "other_frag": reduced
    })

In [133]:
from tqdm.notebook import tqdm
import warnings

In [134]:
all_generated = []
for bait in tqdm(data["bait_frag"].unique()):
    with warnings.catch_warnings(action="ignore"):
        all_generated.append(
            generate_random_negatives(data, bait)
        )

  0%|          | 0/13412 [00:00<?, ?it/s]

In [136]:
all_generated_df = pd.concat(all_generated)

In [139]:
all_generated_df.to_csv("data/intermediates/random_negatives.csv")